In [1]:
import pandas as pd
import json
import gzip
import numpy as np
import random
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise import SVD,SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

from surprise.model_selection import GridSearchCV
from tqdm import tqdm
import matplotlib.pyplot as plt
from surprise.model_selection.validation import cross_validate

C:\Users\pruth\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pruth\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\pruth\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv(r"C:\Users\pruth\Downloads\final project\baseline\datasets\google_reviews.csv")

In [3]:
df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,111435346941066248277,Jenn Thomas,1602893531994,5,"When knowledge is key and kindness matters, Ni...",NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
1,103613723635264236030,Michelle Barta,1575991509006,5,The entire team is outstanding! They are profe...,NaN,"{'time': 1580484156696, 'text': 'We are lucky ...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
2,113556685435236755030,Bethany Baldes,1626981484302,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
3,102235352482458236558,nichole vogt,1583381337082,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
4,114021734869233847378,Kristi Friday,1571112557173,4,NaN,NaN,"{'time': 1571246498430, 'text': 'Thanks for th...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from keras import regularizers
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [6]:
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF  # pure data, 
from libreco.evaluation import evaluate


Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\pruth\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [7]:
df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,111435346941066248277,Jenn Thomas,1602893531994,5,"When knowledge is key and kindness matters, Ni...",NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
1,103613723635264236030,Michelle Barta,1575991509006,5,The entire team is outstanding! They are profe...,NaN,"{'time': 1580484156696, 'text': 'We are lucky ...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
2,113556685435236755030,Bethany Baldes,1626981484302,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
3,102235352482458236558,nichole vogt,1583381337082,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
4,114021734869233847378,Kristi Friday,1571112557173,4,NaN,NaN,"{'time': 1571246498430, 'text': 'Thanks for th...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b


In [8]:
df["user"] = df["user_id"]
df["label"] = df["rating"]
#, "item", "label", "time"
df["item"] = df["gmap_id"]
df=df[["user","item","label"]]

In [9]:
train_data, eval_data, test_data = random_split(df, multi_ratios=[0.8, 0.1, 0.1])


In [10]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [11]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=10,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

In [12]:
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

# do final evaluation on test data
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)

Training start time: 2023-10-28 15:56:47
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\pruth\anaconda3\lib\site-packages\libreco\layers\dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
2023-10-28 15:56:47,319 - WARNING - From C:\Users\pruth\anaconda3\lib\site-packages\keras\layers\normalization\batch_normalization.py:514: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
C:\Users\pruth\anaconda3\lib\site-packages\libreco\layers\dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In parti

Epoch 1 elapsed: 3.097s
	 train_loss: 3.9678


eval_pointwise: 100%|███████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 120.00it/s]


	 eval rmse: 1.0836


...

Epoch 10 elapsed: 1.649s
	 train_loss: 0.3989


eval_pointwise: 100%|███████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 374.97it/s]


	 eval rmse: 1.0420


eval_pointwise: 100%|███████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 386.87it/s]


{'loss': 1.0395138}

In [13]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["mae"],
)

eval_pointwise: 100%|███████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 315.77it/s]


{'mae': 0.7465751}

In [14]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["rmse"],
)

eval_pointwise: 100%|███████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 352.82it/s]


{'rmse': 1.0395138}